In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers

In [3]:
(train_data,train_label),(test_data,test_label)=mnist.load_data()

print(f'The Train Data shape : {train_data.shape}')
print(f'The Test Data shape : {test_data.shape}')

11501568/11490434 [==============================] - 0s 0us/step
The Train Data shape : (60000, 28, 28)
The Test Data shape : (10000, 28, 28)


In [4]:
train_data = train_data.reshape(-1,28,28,1).astype('float32')/255
test_data = test_data.reshape(-1,28,28,1).astype('float32')/255

train_label = to_categorical(train_label)
test_label = to_categorical(test_label)

train_data,val_data,train_label,val_label=train_test_split(train_data,train_label,test_size=0.25,random_state=13)

In [5]:
train_datagen=ImageDataGenerator(rotation_range=15 , 
                             width_shift_range=0.2 , 
                             height_shift_range=0.2 ,
                             shear_range=0.2 ,
                             zoom_range=0.2 , 
                             data_format='channels_last')


train_datagen.fit(train_data)

LR_function=ReduceLROnPlateau(monitor='val_acc',
                             patience=3,
                             # 3 epochs 內acc沒下降就要調整LR
                             verbose=1,
                             factor=0.5,
                             # LR降為0.5
                             min_lr=0.00001
                             # 最小 LR 到0.00001就不再下降
                             )

In [6]:
#進行 CNN 模型的建構
model=models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

model.compile(optimizer='Adam' , loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_datagen.flow(train_data,train_label,batch_size=64), 
                              steps_per_epoch=train_data.shape[0]/64 , epochs=100,validation_data=(val_data,val_label),validation_steps=val_data.shape[0]/64,callbacks=[LR_function])

In [7]:
history = model.fit_generator(train_datagen.flow(train_data,train_label,batch_size=64), 
                              steps_per_epoch=train_data.shape[0]/64 , epochs=100,validation_data=(val_data,val_label),validation_steps=val_data.shape[0]/64,callbacks=[LR_function])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/100
703/703 [==============================] - 52s 72ms/step - loss: 0.8500 - accuracy: 0.7129 - val_loss: 0.1436 - val_accuracy: 0.9576 - lr: 0.0010
Epoch 2/100
703/703 [==============================] - 50s 72ms/step - loss: 0.3487 - accuracy: 0.8928 - val_loss: 0.1018 - val_accuracy: 0.9687 - lr: 0.0010
Epoch 3/100
703/703 [==============================] - 51s 72ms/step - loss: 0.2662 - accuracy: 0.9165 - val_loss: 0.0687 - val_accuracy: 0.9775 - lr: 0.0010


KeyboardInterrupt: ignored

In [ ]:
loss,acc=model.evaluate(test_data,test_label)
print(f'The Accuracy on Test Data = {acc}')

In [ ]:
acc = history.history['acc']
loss = history.history['loss']
val_acc=history.history['val_acc']
val_loss=history.history['val_loss']

epochs=range(100)

plt.plot(epochs,acc,'r-',label='Training Accuracy')
plt.plot(epochs,val_acc,'b-',label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs,loss,'r-',label='Training Loss')
plt.plot(epochs,val_loss,'b-',label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()